In [209]:
#!cp 3-one_hot.py 4-train.py
!chmod +x *.py

In [99]:
#!/usr/bin/env python3

#build_model = __import__('0-sequential').build_model
""" doc """
import tensorflow.keras as K


def build_model(nx, layers, activations, lambtha, keep_prob):
    """ doc """
    model = K.Sequential()
    for i in range(len(layers)):
        init = K.initializers.VarianceScaling(mode="fan_avg")
        # freg = K.layers.ActivityRegularization(l2=lambtha)
        freg = K.regularizers.l2(lambtha)
        layer = K.layers.Dense(layers[i], input_dim=nx,
                               activation=activations[i],
                               kernel_initializer=init,
                               kernel_regularizer=freg)
        model.add(layer)
        if i != len(layers)-1:
            dropped = K.layers.Dropout(rate=1-keep_prob)
            model.add(dropped)
    return (model)


if __name__ == '__main__':
    network = build_model(784, [256, 256, 10], ['tanh', 'tanh', 'softmax'], 0.001, 0.95)
    network.summary()
    print(network.losses)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_197 (Dense)            (None, 256)               200960    
_________________________________________________________________
dropout_104 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_198 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_105 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_199 (Dense)            (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________
[<tf.Tensor 'kernel/Regularizer_149/add:0' shape=() dtype=float32>, <tf.Tensor 'kernel/Regularizer_150/add:0' shape=() dtype=float32>, <

In [191]:
def build_model(nx, layers, activations, lambtha, keep_prob):
    inputs = K.Input(shape=(nx,))
    freg = K.regularizers.l2(lambtha)
    x = K.layers.Dense(layers[0], activation=activations[0],
                           kernel_regularizer=freg)(inputs)
    for i in range(1, len(layers)):
        x = K.layers.Dropout(rate=1-keep_prob)(x)
        x = K.layers.Dense(layers[i], activation=activations[i],
                           kernel_regularizer=freg)(x)
        
    model = tf.keras.Model(inputs=inputs, outputs=x)
    return (model)

if __name__ == '__main__':
    network = build_model(784, [256, 256, 10], ['tanh', 'tanh', 'softmax'], 0.001, 0.95)
    network.summary()
    print(network.losses)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_103 (InputLayer)       (None, 784)               0         
_________________________________________________________________
dense_409 (Dense)            (None, 256)               200960    
_________________________________________________________________
dropout_309 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_410 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_310 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_411 (Dense)            (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________
[<tf

In [204]:
import tensorflow as tf

build_model = __import__('1-input').build_model
# optimize_model = __import__('2-optimize').optimize_model
def optimize_model(network, alpha, beta1, beta2):
    """ doc """
    optimizer = K.optimizers.Adam(alpha, beta1, beta2)
    network.compile(optimizer,
                    loss="categorical_crossentropy",
                    metrics=['accuracy'])
    return None
if __name__ == '__main__':
    model = build_model(784, [256, 256, 10], ['tanh', 'tanh', 'softmax'], 0.001, 0.95)
    optimize_model(model, 0.01, 0.99, 0.9)
    print(model.loss)
    print(model.metrics)
    opt = model.optimizer
    print(opt.__class__)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print(sess.run((opt.lr, opt.beta_1, opt.beta_2))) 

categorical_crossentropy
['accuracy']
<class 'tensorflow.python.keras.optimizers.Adam'>
(0.01, 0.99, 0.9)


In [207]:
#!/usr/bin/env python3

import numpy as np
#one_hot = __import__('3-one_hot').one_hot
def one_hot(labels, classes=None):
    """ doc """
    return K.utils.to_categorical(labels, classes)

if __name__ == '__main__':
    labels = np.load('../data/MNIST.npz')['Y_train'][:10]
    print(labels)
    print(one_hot(labels))   

[5 0 4 1 9 2 1 3 1 4]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [ ]:
#!/usr/bin/env python3

import numpy as np
import tensorflow as tf
build_model = __import__('1-input').build_model
optimize_model = __import__('2-optimize').optimize_model
one_hot = __import__('3-one_hot').one_hot
#train_model = __import__('4-train').train_model

def train_model(network, data, labels, batch_size,
                epochs, verbose=True, shuffle=False):
    """ doc """
    if verbose == True:
        verbose = 1
    else:
        verbose = 0
    return network.fit(data, labels,
                       batch_size=batch_size,
                       epochs=epochs,
                       verbose=verbose,
                       shuffle=shuffle)
if __name__ == '__main__':
    datasets = np.load('../data/MNIST.npz')
    X_train = datasets['X_train']
    X_train = X_train.reshape(X_train.shape[0], -1)
    Y_train = datasets['Y_train']
    Y_train_oh = one_hot(Y_train)

    np.random.seed(0)
    tf.set_random_seed(0)
    lambtha = 0.0001
    keep_prob = 0.95
    network = build_model(784, [256, 256, 10], ['relu', 'relu', 'softmax'], lambtha, keep_prob)
    alpha = 0.001
    beta1 = 0.9
    beta2 = 0.999
    optimize_model(network, alpha, beta1, beta2)
    batch_size = 64
    epochs = 5
    train_model(network, X_train, Y_train_oh, batch_size, epochs)


Epoch 1/5
50000/50000 [==============================] - 15s 300us/step - loss: 0.3341 - acc: 0.9182
Epoch 2/5
50000/50000 [==============================] - 11s 219us/step - loss: 0.1772 - acc: 0.9649
Epoch 3/5
42240/50000 [========================>.....] - ETA: 2s - loss: 0.1427 - acc: 0.9752